In [2]:
import os
import requests
import numpy as np
import pandas as pd
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from scipy import optimize
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import glob
import json

In [3]:
#expirimenting with USA Spending API calls
payload = {'fiscal_year':'2017', 'funding_agency_id':'766'}


url = 'https://api.usaspending.gov/api/v2/federal_obligations/'
response = requests.get(url, params = payload)
data = response.content #byte object
data_str = data.decode('UTF-8')
obligations_dict = json.loads(data_str)
obligations_df = pd.DataFrame(obligations_dict['results'])


url2 = 'https://api.usaspending.gov/api/v2/financial_spending/major_object_class/'
response2 = requests.get(url2, params = payload)
data2 = response2.content #byte object
data2_str = data2.decode('UTF-8')
obligations_dict2 = json.loads(data2_str)
obligations_df2 = pd.DataFrame(obligations_dict2['results'])




# response3 = requests.get('https://api/v2/financial_spending/object_class/?fiscal_year=2017&funding_agency_id=4324&major_object_class_code=20')
# print(type(response.content))
# print(response.content)
# print('*****************************')
# print(response2.content)
# print(obligations_dict.keys())

obligations_df2.head()

,major_object_class_code,major_object_class_name,obligated_amount
0,00,Unknown Object Type,0.00
1,10,Personnel compensation and benefits,31126608505.51
2,20,Contractual services and supplies,29432960835.15
3,30,Acquisition of assets,4188989649.05
4,40,Grants and fixed charges,17416808670.08


In [4]:
# read all csv files within the test_data folder. This should be 2008-2018 (11 years), of DHS contract award data
all_files = glob.glob("../data/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, header = 0, usecols = ['contract_transaction_unique_key',
                        'contract_award_unique_key',
                        'federal_action_obligation','total_dollars_obligated', 'base_and_exercised_options_value', 
                        'current_total_value_of_award', 'base_and_all_options_value','potential_total_value_of_award',
                        'action_date'],
                 dtype = {'contract_transaction_unique_key':'str','contract_transaction_unique_key':'str',
                          'federal_action_obligation': 'float','total_dollars_obligated': 'float', 
                          'base_and_exercised_options_value': 'float', 'current_total_value_of_award': 'float', 
                          'base_and_all_options_value': 'float','potential_total_value_of_award': 'float',
                          'action_date':'str'})
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)


In [5]:
df.head()

,contract_transaction_unique_key,contract_award_unique_key,federal_action_obligation,total_dollars_obligated,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award,action_date
0,7014_-NONE-_HSBP1010P00497_P00003_-NONE-_0,CONT_AWD_HSBP1010P00497_7014_-NONE-_-NONE-,0.00,NaN,0.00,NaN,0.00,NaN,2010-09-30
1,7013_8000_HSTS0310JCIO071_0_NNG07DA46B_0,CONT_AWD_HSTS0310JCIO071_7013_NNG07DA46B_8000,3885103.94,NaN,3885103.94,NaN,10223807.69,NaN,2010-09-30
2,7022_7022_HSFEHQ04J0056_P00003_HSFEHQ04D0126_0,CONT_AWD_HSFEHQ04J0056_7022_HSFEHQ04D0126_7022,-76943.80,NaN,-76943.80,NaN,-76943.80,NaN,2010-09-30
3,7022_4730_HSFEMW09F0066_P00002_GS35F0667K_0,CONT_AWD_HSFEMW09F0066_7022_GS35F0667K_4730,0.00,NaN,0.00,NaN,0.00,NaN,2010-09-30
4,7014_7014_HSBP1010J00870_0_HSBP1010D00023_0,CONT_AWD_HSBP1010J00870_7014_HSBP1010D00023_7014,99990.00,NaN,99990.00,NaN,99990.00,NaN,2010-09-30


In [6]:
# This is what we are using for our rate of Execution
df['Percent Cumulatively Obligated over potential total value of award'] = df['total_dollars_obligated'] / df['potential_total_value_of_award']

# Create Indicator Variable
df['Indicator'] = df['federal_action_obligation']<-1000

##Separate contracts that have a deobligation

In [7]:
#Separate contracts that have a deobligation
df_deob = df.copy().loc[df['Indicator'] == 1]

In [8]:
#Get first deobligation date of contracts

df_first_deob = df_deob.copy()
df_first_deob = df_first_deob[['contract_award_unique_key', 'action_date']]
df_first_deob = df_first_deob.groupby(['contract_award_unique_key'], as_index = False).min()
df_first_deob.rename(columns = {'action_date':'first_deob_date'}, inplace = True)
df_first_deob['first_deob_date'] = pd.to_datetime(df_first_deob['first_deob_date'])


In [9]:
#Add indicator flag for contracts that ever have deobligation
df_deob.drop(columns = [c for c in df_deob.columns if c not in ['Indicator', 'contract_award_unique_key']], inplace = True)
df_deob = df_deob.drop_duplicates()
df_deob.rename(columns = {'Indicator': 'Contract_deob'}, inplace = True)
df_ever_deob = df.merge(df_deob, how = 'left', right_on = 'contract_award_unique_key', left_on = 'contract_award_unique_key')
df_ever_deob.rename(columns = {'Indicator': 'Transaction_Deob'}, inplace = True)
df_ever_deob['Contract_deob'] = df_ever_deob['Contract_deob'].fillna(False)
df_ever_deob.head()

,contract_transaction_unique_key,contract_award_unique_key,federal_action_obligation,total_dollars_obligated,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award,action_date,Percent Cumulatively Obligated over potential total value of award,Transaction_Deob,Contract_deob
0,7014_-NONE-_HSBP1010P00497_P00003_-NONE-_0,CONT_AWD_HSBP1010P00497_7014_-NONE-_-NONE-,0.00,NaN,0.00,NaN,0.00,NaN,2010-09-30,NaN,False,False
1,7013_8000_HSTS0310JCIO071_0_NNG07DA46B_0,CONT_AWD_HSTS0310JCIO071_7013_NNG07DA46B_8000,3885103.94,NaN,3885103.94,NaN,10223807.69,NaN,2010-09-30,NaN,False,True
2,7022_7022_HSFEHQ04J0056_P00003_HSFEHQ04D0126_0,CONT_AWD_HSFEHQ04J0056_7022_HSFEHQ04D0126_7022,-76943.80,NaN,-76943.80,NaN,-76943.80,NaN,2010-09-30,NaN,True,True
3,7022_4730_HSFEMW09F0066_P00002_GS35F0667K_0,CONT_AWD_HSFEMW09F0066_7022_GS35F0667K_4730,0.00,NaN,0.00,NaN,0.00,NaN,2010-09-30,NaN,False,False
4,7014_7014_HSBP1010J00870_0_HSBP1010D00023_0,CONT_AWD_HSBP1010J00870_7014_HSBP1010D00023_7014,99990.00,NaN,99990.00,NaN,99990.00,NaN,2010-09-30,NaN,False,False


In [10]:
#Merge first deobligation date
contract_df = df_ever_deob.merge(df_first_deob, how = 'left', right_on = 'contract_award_unique_key', left_on = 'contract_award_unique_key')


In [11]:
#test cell, there should be both types of values in both indicator columns
print(contract_df['Contract_deob'].value_counts())
print(contract_df['Transaction_Deob'].value_counts())
print(df_deob.shape)

False    780736
True     306051
Name: Contract_deob, dtype: int64
False    1003532
True       83255
Name: Transaction_Deob, dtype: int64
(70072, 2)


In [12]:
#removing rows after first deobligation
contract_df['action_date'] = pd.to_datetime(contract_df['action_date'])
contract_df['first_deob_date'] = pd.to_datetime(contract_df['first_deob_date'])
contract_df = contract_df[contract_df['action_date'] <= contract_df['first_deob_date']]

In [13]:
#Test cell
dftest3 = contract_df[contract_df['contract_award_unique_key'] == 'CONT_AWD_HSTS0310JCIO071_7013_NNG07DA46B_8000'].sort_values(by = 'action_date')

dftest3.head(10)

,contract_transaction_unique_key,contract_award_unique_key,federal_action_obligation,total_dollars_obligated,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award,action_date,Percent Cumulatively Obligated over potential total value of award,Transaction_Deob,Contract_deob,first_deob_date
1,7013_8000_HSTS0310JCIO071_0_NNG07DA46B_0,CONT_AWD_HSTS0310JCIO071_7013_NNG07DA46B_8000,3885103.94,NaN,3885103.94,NaN,10223807.69,NaN,2010-09-30,NaN,False,True,2012-06-21
217634,7013_8000_HSTS0310JCIO071_P00001_NNG07DA46B_0,CONT_AWD_HSTS0310JCIO071_7013_NNG07DA46B_8000,3344677.82,NaN,3344677.82,NaN,0.00,NaN,2011-06-16,NaN,False,True,2012-06-21
133927,7013_8000_HSTS0310JCIO071_P00002_NNG07DA46B_0,CONT_AWD_HSTS0310JCIO071_7013_NNG07DA46B_8000,2994025.93,NaN,2994025.93,NaN,0.00,NaN,2012-05-04,NaN,False,True,2012-06-21
123103,7013_8000_HSTS0310JCIO071_P00003_NNG07DA46B_0,CONT_AWD_HSTS0310JCIO071_7013_NNG07DA46B_8000,-402426.96,NaN,-402426.96,NaN,-402426.96,NaN,2012-06-21,NaN,True,True,2012-06-21


In [14]:
#Get first action date of all contracts

df_first_date = contract_df.copy()
df_first_date = df_first_date[['contract_award_unique_key', 'action_date']]
df_first_date = df_first_date.groupby(['contract_award_unique_key'] , as_index = False).min()
df_first_date.rename(columns = {'action_date': 'first_date'}, inplace = True)
df_first_date.head()

,contract_award_unique_key,first_date
0,CONT_AWD_00003_7022_HSFEHQ08D0105_7022,2009-03-18
1,CONT_AWD_00004_7022_HSFE0408A0017_7022,2008-05-14
2,CONT_AWD_0001_2036_TPDFIGBPA100001_2036,2010-10-01
3,CONT_AWD_0001_7022_HSFE0609A6334_7022,2009-03-13
4,CONT_AWD_0001_7022_HSFEHQ09A0170_7022,2008-12-31


In [15]:
#test cell
df_test_first_date = df_first_date.copy()

df_test_first_date = df_first_date[df_first_date['contract_award_unique_key'] == 'CONT_AWD_HSTS0310JCIO071_7013_NNG07DA46B_8000']
df_test_first_date.head()


,contract_award_unique_key,first_date
67075,CONT_AWD_HSTS0310JCIO071_7013_NNG07DA46B_8000,2010-09-30


In [16]:
contract_df = contract_df.merge(df_first_date, how = 'left', right_on = 'contract_award_unique_key', left_on = 'contract_award_unique_key')
contract_df['action_date'] = pd.to_datetime(contract_df['action_date'])
contract_df['first_date'] = pd.to_datetime(contract_df['first_date'])
contract_df['days_since_first_action'] = contract_df['action_date'] - contract_df['first_date']

In [17]:
#test cell
contract_df_test = contract_df.copy()
contract_df_test = contract_df_test[contract_df_test['contract_award_unique_key'] == 'CONT_AWD_HSTS0310JCIO071_7013_NNG07DA46B_8000'].sort_values(by = 'action_date')
contract_df_test.head()

,contract_transaction_unique_key,contract_award_unique_key,federal_action_obligation,total_dollars_obligated,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award,action_date,Percent Cumulatively Obligated over potential total value of award,Transaction_Deob,Contract_deob,first_deob_date,first_date,days_since_first_action
0,7013_8000_HSTS0310JCIO071_0_NNG07DA46B_0,CONT_AWD_HSTS0310JCIO071_7013_NNG07DA46B_8000,3885103.94,NaN,3885103.94,NaN,10223807.69,NaN,2010-09-30,NaN,False,True,2012-06-21,2010-09-30,0 days
57279,7013_8000_HSTS0310JCIO071_P00001_NNG07DA46B_0,CONT_AWD_HSTS0310JCIO071_7013_NNG07DA46B_8000,3344677.82,NaN,3344677.82,NaN,0.00,NaN,2011-06-16,NaN,False,True,2012-06-21,2010-09-30,259 days
35134,7013_8000_HSTS0310JCIO071_P00002_NNG07DA46B_0,CONT_AWD_HSTS0310JCIO071_7013_NNG07DA46B_8000,2994025.93,NaN,2994025.93,NaN,0.00,NaN,2012-05-04,NaN,False,True,2012-06-21,2010-09-30,582 days
32386,7013_8000_HSTS0310JCIO071_P00003_NNG07DA46B_0,CONT_AWD_HSTS0310JCIO071_7013_NNG07DA46B_8000,-402426.96,NaN,-402426.96,NaN,-402426.96,NaN,2012-06-21,NaN,True,True,2012-06-21,2010-09-30,630 days


In [18]:

#Contract_df now has the contract/transaction deobligation flags and days since the first action date

In [47]:
# Adding in transaction number
g = contract_df.groupby(['contract_award_unique_key'])

contract_df['transaction_num'] = g['action_date'].rank(method = 'min')



In [49]:
#test cell
testdf = contract_df.copy()
testdf = testdf[testdf['contract_award_unique_key'] == 'CONT_AWD_HSTS0310JCIO071_7013_NNG07DA46B_8000'].sort_values(by = 'transaction_num')
testdf.head()

,contract_transaction_unique_key,contract_award_unique_key,federal_action_obligation,total_dollars_obligated,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award,action_date,Percent Cumulatively Obligated over potential total value of award,Transaction_Deob,Contract_deob,first_deob_date,first_date,days_since_first_action,transaction_num
0,7013_8000_HSTS0310JCIO071_0_NNG07DA46B_0,CONT_AWD_HSTS0310JCIO071_7013_NNG07DA46B_8000,3885103.94,NaN,3885103.94,NaN,10223807.69,NaN,2010-09-30,NaN,False,True,2012-06-21,2010-09-30,0 days,1.0
57279,7013_8000_HSTS0310JCIO071_P00001_NNG07DA46B_0,CONT_AWD_HSTS0310JCIO071_7013_NNG07DA46B_8000,3344677.82,NaN,3344677.82,NaN,0.00,NaN,2011-06-16,NaN,False,True,2012-06-21,2010-09-30,259 days,2.0
35134,7013_8000_HSTS0310JCIO071_P00002_NNG07DA46B_0,CONT_AWD_HSTS0310JCIO071_7013_NNG07DA46B_8000,2994025.93,NaN,2994025.93,NaN,0.00,NaN,2012-05-04,NaN,False,True,2012-06-21,2010-09-30,582 days,3.0
32386,7013_8000_HSTS0310JCIO071_P00003_NNG07DA46B_0,CONT_AWD_HSTS0310JCIO071_7013_NNG07DA46B_8000,-402426.96,NaN,-402426.96,NaN,-402426.96,NaN,2012-06-21,NaN,True,True,2012-06-21,2010-09-30,630 days,4.0


## End Mike's changes 05/14/2020

In [ ]:
#Test cell
## We need to ask Eric about these to see if the numbers make sense

df_test4 = contract_df[contract_df['base_and_all_options_value'] != contract_df['federal_action_obligation']]
print(contract_df.shape)
#6,519,296 rows in df

print(df_test4.shape)
#335,073 where federal action obligatio != base and exercised options value
#352,596 where base & all options != base and exercised options
#300,646 where base & all options value != federal action obligation

df_test4.head()

In [ ]:
df = df.fillna(0)
# Convert action date to pd.datetime
df['action_date'] = pd.to_datetime(df['action_date'])
# set index to each contract and date
df.set_index(['contract_award_unique_key','action_date'], inplace = True)
df.sort_index()

In [ ]:
# We want to only focus on execution rate (for now). I'm not sure which column is the best indicator of execution rate
# we can follow up with Eric. I'm going to stick with Percent Cumulatively Obligated over potential total value of award
# as wequal to the execution rate.
drop_columns = [column for column in df.columns if column != 'Percent Cumulatively Obligated over potential total value of award']
df.drop(drop_columns,axis = 1, inplace=True)

df.rename(columns = {'Percent Cumulatively Obligated over potential total value of award':'execution_rate'},inplace = True)

In [ ]:
df.head(4)

In [ ]:
contracts = [contract for contract, date in list(df.index)]
unique_k = set(contracts)
print('This is the number of unique Contracts: ' + str(len(unique_k)))
print('This is the number of transactions: ' + str(len(contracts)))

# Selecting contracts w/ 5 or more transactions from 2008-2018

In [ ]:
# I grouped the dataframe by contract and used the .count(method)
# to determine the number of entries. I then used an inequality to get a boolean series of the number of contracts 
# that satisfied the condition of at least 5 transactions.
test = df.groupby(level='contract_award_unique_key')['execution_rate'].count() >= 5
test = test[test==True]
# This is the list of contracts that have more than 5 transactions. 
highlighted_transactions = test.index



### Utilize FY19 Data as Test Data

In [ ]:
os.chdir('C:\\Users\\belincoln\\repos\\BudgetPredict')

In [ ]:
%cd data

In [ ]:
filename = 'FY2019_070_Contracts_Full_20200110_1.csv'

In [ ]:
test_df = pd.read_csv(filename, header = 0, usecols = ['contract_transaction_unique_key',
                        'contract_award_unique_key',
                        'federal_action_obligation','total_dollars_obligated', 'base_and_exercised_options_value', 
                        'current_total_value_of_award', 'base_and_all_options_value','potential_total_value_of_award',
                        'action_date'],
                 dtype = {'contract_transaction_unique_key':'str','contract_transaction_unique_key':'str',
                          'federal_action_obligation': 'float','total_dollars_obligated': 'float', 
                          'base_and_exercised_options_value': 'float', 'current_total_value_of_award': 'float', 
                          'base_and_all_options_value': 'float','potential_total_value_of_award': 'float',
                          'action_date':'str'})

In [ ]:
print('This is the # of transactions is FY19 for DHS: ' + str(len(test_df)))

In [ ]:
test_df = test_df.fillna(0)
# Convert action date to pd.datetime
test_df['action_date'] = pd.to_datetime(test_df['action_date'])
# set index to each contract and date
test_df.set_index(['contract_award_unique_key','action_date'], inplace = True)
test_df.sort_index()
fy19_df = test_df.copy()

In [ ]:
# We want to only focus on execution rate (for now). I'm not sure which column is the best indicator of execution rate
# we can follow up with Eric. I'm going to stick with Percent Cumulatively Obligated over potential total value of award
# as wequal to the execution rate.
drop_columns = [column for column in test_df.columns if column != 'Percent Cumulatively Obligated over potential total value of award']
test_df.drop(drop_columns,axis = 1, inplace=True)

test_df.rename(columns = {'Percent Cumulatively Obligated over potential total value of award':'execution_rate'},inplace = True)

In [ ]:
contracts = [contract for contract, date in list(test_df.index)]
unique_k_fy19 = set(contracts)
print('This is the number of unique Contracts: ' + str(len(unique_k_fy19)))
print('This is the number of transactions: ' + str(len(contracts)))

In [ ]:
type(unique_k_fy19)

In [ ]:
highlighted_transactions = set(highlighted_transactions)

In [ ]:
len(unique_k_fy19.intersection(highlighted_transactions))

In [ ]:
mask = list(unique_k_fy19.intersection(highlighted_transactions))

In [ ]:
fy19_df.index.get_level_values('contract_award_unique_key')

In [ ]:
highlighted_fy19 = fy19_df.loc[(list(highlighted_transactions), slice(None)),:]

In [ ]:
highlighted_fy19.head()

In [ ]:
highlighted_fy19.index.get_level_values(0).unique()

In [ ]:
highlighted_fy19

In [ ]:
highlighted_fy19.drop('base_and_exercised_options_value',axis = 1, inplace = True)

In [ ]:
df = highlighted_fy19

In [ ]:
# Create 3 new features for analysis
df['Percent awarded over potential total awarded'] = df['current_total_value_of_award'] / df['potential_total_value_of_award']
df['Percent Cumulatively Obligated over potential total value of award'] = df['total_dollars_obligated'] / df['potential_total_value_of_award']
df['Percent Cumulatively Obligated over total value already awarded'] = df['total_dollars_obligated'] / df['current_total_value_of_award']

# Create Indicator Variable
df['Indicator'] = df['federal_action_obligation']<-1000

# set index to each transaction key
df.set_index('contract_transaction_unique_key', inplace = True)

In [ ]:
df.drop('federal_action_obligation', axis =1, inplace = True)
df = df.fillna(0)

In [ ]:
# This is the percentage of De-Obligations in the sample (before test/train split)
print('Total number of De-Obligations: ' +str(df['Indicator'].sum()))
print('Percentage of De-Obligations: '+str(df['Indicator'].sum()/len(df)))

In [ ]:
# Create matrix of feature variables and our dependent variable
X, y =  df.iloc[:,1:-1], df.loc[:,'Indicator']

# convert y to np.array of 0s and 1s
y = np.array(y.astype(int))

In [ ]:
#train the data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [ ]:
X_test.shape

In [ ]:
# Setup the data matrix appropriately, and add ones for the intercept term
m_train, n_train = X_train.shape
m_test, n_test = X_test.shape
# Add intercept term to X
X_train = np.concatenate([np.ones((m_train, 1)), X_train], axis=1)
X_test = np.concatenate([np.ones((m_test, 1)), X_test], axis=1)



In [ ]:
log_model = LogisticRegression().fit(X_train, y_train)

In [ ]:
# Coefficents to multiply features by
log_model.coef_

In [ ]:
theta = log_model.coef_[0]

In [ ]:
print(theta[0])

In [ ]:
def sigmoid(z):
    """
    Compute sigmoid function given the input z.
    
    Parameters
    ----------
    z : array_like
        The input to the sigmoid function. This can be a 1-D vector 
        or a 2-D matrix. 
    
    Returns
    -------
    g : array_like
        The computed sigmoid function. g has the same shape as z, since
        the sigmoid is computed element-wise on z.
        
    Instructions
    ------------
    Compute the sigmoid of each value of z (z can be a matrix, vector or scalar).
    """
    # convert input to a numpy array
    z = np.array(z)
    
    # You need to return the following variables correctly 
    g = np.zeros(z.shape)

    temp = 1 + np.power(np.e,-z)
    g = 1 / temp
    

    return g

In [ ]:
pred = sigmoid(np.dot(X_test,theta))

In [ ]:
fig = pyplot.figure
pyplot.scatter(np.arange(len(pred)), pred)

In [ ]:
# Now we will test our model using the test_set
pred = sigmoid(np.dot(X_test,theta))

In [ ]:
# Convert probabilty of de-obligation into prediction
pred = pred > .5
# Convert to int
pred = np.array(pred.astype(int))

In [ ]:
test = pred == y_test

In [ ]:
test.sum()/len(test)

In [ ]:
# Really what we are looking for here 
# is how many times a de-obligation occured and we were able to predict it?

In [ ]:
# Also percentage of false positives: how many times did we predict a de-obligation and 
# it did not occur?

In [ ]:
data = {'predicted value' : pred, 'test value': y_test}
testdf = pd.DataFrame(data)

In [ ]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

In [ ]:


cm = np.array(confusion_matrix(pred, y_test, labels = [1,0]))
confusion = pd.DataFrame(cm, index=[['Predicted', 'Predicted'], ['Large De-Obligation','Obligation']],
                         columns = [['Test Results', 'Test Results'],['Large De-Obligation','Obligation']])
confusion


In [ ]:
confusion.iloc[0].sum()+confusion.iloc[1].sum()

In [ ]:
correct_preds = testdf[testdf['test value'] == 1]
correct_preds2 = correct_preds[correct_preds['predicted value'] ==1]
print('Size of Test set: ' + str(len(testdf)))
print('Size of Train set: ' + str(len(X_train)))
print('Percentage of correct predictions (Accuracey): ' + str((confusion.iloc[0,0] + confusion.iloc[1,1])/len(testdf)))
print('Percentage of significant de-obligations: ' + str(len(correct_preds)/len(testdf)))
print('Percentage of de-obligations correctly predicted: ' + str(len(correct_preds2)/len(correct_preds)))

In [ ]:
# Rate of False Negatives: A de-obligation occurs, but the model did not predict it. 
false_neg = testdf[testdf['predicted value'] == 0]
false_neg2 = false_neg[false_neg['test value'] == 1]
print('Number of False Negatives : ' + str(len(false_neg2)))
print('Rate of False Negative: ' + str(len(false_neg2)/len(testdf)))

In [ ]:
# Show the rate of False Positives: Times the model predicted a de-obligation, yet there wasn't one

false_pos = testdf[testdf['predicted value'] == 1]
false_pos2 = false_pos[false_pos['test value'] == 0]

print('Number of False Positives: ' + str(len(false_pos2)))
print('Rate of False Positives: ' + str(len(false_pos2)/len(testdf)))